In [1]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
# !gdown ...

!noglob gdown --fuzzy https://drive.google.com/file/d/1vC5tuPSR769P3n5iHkVTKL6yclJW0l5X/view?usp=sharing -O data-vn.csv
# !noglob gdown --fuzzy https://drive.google.com/file/d/1oSXQHLoVSGfBOLR4NjNwQRTkDb8Zd8OU/view?usp=sharing -O list_province.txt
# !noglob gdown --fuzzy https://drive.google.com/file/d/18sZoDAqJWyUfmjQN3VpKfkDHFQ-tcml6/view?usp=sharing -O list_district.txt
# !noglob gdown --fuzzy https://drive.google.com/file/d/1VfDCj7R11jf3SIZyoZdYL7fIN-AIhC-1/view?usp=sharing -O list_ward.txt

Downloading...
From: https://drive.google.com/uc?id=1vC5tuPSR769P3n5iHkVTKL6yclJW0l5X
To: /home/phuocho/Projects/advance_algorithm/data-vn.csv
100%|██████████████████████████████████████| 5.82M/5.82M [00:00<00:00, 72.7MB/s]


In [2]:
# NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
# !pip install pandas
# !pip install Unidecode

In [3]:
# NOTE: you CAN change this cell
# import your library here
import time
import re
import heapq
import pandas as pd
from concurrent.futures import TimeoutError
from unidecode import unidecode
import logging


logging.basicConfig(
    filemode='w',
    filename='address_.log',
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S',
)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.

class Solution:
    
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'

        self.list_province = set(pd.read_csv(self.province_path, sep=',', header=None).iloc[:,0].values.tolist())
        self.list_district = set(pd.read_csv(self.district_path, sep=',', header=None).iloc[:,0].values.tolist())
        self.list_ward = set(pd.read_csv(self.ward_path, sep=',', header=None).iloc[:,0].values.tolist())

        # write your preprocess here, add more method if needed
        self.trie = TrieNode()

        self.trie2 = TrieNode()

        cities_df = pd.read_csv('data-vn.csv', sep=',', header=0)
        cities_df.fillna('', inplace=True)
        data = cities_df[['combination', 'ward', 'district', 'province']].values.tolist()
        # data.extend(self._construct_data_2())
        self.trie.build(data)

        # self.trie2.build(self._construct_data_2())

    def _construct_data_2(self):
        list_province = pd.read_csv('list_province.txt', sep=',', header=None).iloc[:,0].values.tolist()
        list_district = pd.read_csv('list_district.txt', sep=',', header=None).iloc[:,0].values.tolist()
        list_ward = pd.read_csv('list_ward.txt', sep=',', header=None).iloc[:,0].values.tolist()

        list = []
        # for ward in list_ward:
        #   for district in list_district:
        #       for province in list_province:
        #           list.append(
        #                 (' '.join([ward, district, province]), ward, district, province)
        #               )
        for district in list_district:
          for province in list_province:
              list.append(
                    (' '.join(["", district, province]), "", district, province)
                  )
              list.append(
                    (' '.join(["", "", province]), "", "", province)
                  )
        return list
    
    def process(self, s: str):
      
        try:
          return self.find_ans(s)
        except OSError:
          return {"province": "", "district": "", "ward": ""}
        except TimeoutError:
          return {"province": "", "district": "", "ward": ""}
        except Exception as e:
          return {"province": "", "district": "", "ward": ""}

    def find_ans(self, s: str):
      # write your process string here
      raw_input = Input(s)
      processed_input = raw_input.clean_noise_input()

      t_start_search = time.perf_counter_ns()

      edit_distance = EditDistance(processed_input, t_start_search)
      areas_results = edit_distance.search(self.trie, 4)
      while not areas_results and processed_input:
        processed_input = remove_word_from_left(processed_input)
        edit_distance = EditDistance(processed_input, t_start_search)
        areas_results = edit_distance.search(self.trie, 4)

      if len(areas_results) > 1:
        areas_results = self._get_top_confuse_from_result(areas_results)

      logger.info(f'Raw input: {s}')
      logger.info(f'Processed input: {raw_input.clean_noise_input()}')
      logger.info(f'areas_results: {areas_results}')
      if areas_results:
        province, district, ward = self._try_guest(s, areas_results)
        
        if len(province) > 0 and  province not in self.list_province:
            province = ""
        if len(district) > 0 and district not in self.list_district:
            district = ""
        if len(ward) > 0 and ward not in self.list_ward:
            ward = ""

        return {
          "province": province,
          "district": district,
          "ward": ward,
        }
      return {"province": "", "district": "", "ward": ""}
  
    def _get_top_confuse_from_result(self, results):
        if len(results) == 0:
            return []
        
        same_edit_distance_list = []
        min_edit_distance = results[0][0]
        for r in results:
            if r[0] == min_edit_distance:
                same_edit_distance_list.append(r)

        return list(set(same_edit_distance_list))

    def _try_guest(self, raw, confuse_list):
        if len(confuse_list) == 1:
            return confuse_list[0][1]
        if confuse_list[0][0] == 0:
          return confuse_list[0][1]
        
        data_list = [d[1] for d in confuse_list]
        
        score_list = []
        raw_normalize = self._nomalize_word(raw)
        for province, district, ward in data_list:
            score = 0
            score += self._get_score(raw_normalize, province)
            score += self._get_score(raw_normalize, district)
            score += self._get_score(raw_normalize, ward)
            heapq.heappush(score_list , (-score, (province, district, ward)))

        logger.info(f'Score list: \t{score_list}')
        confuse_score_list = self._get_top_confuse_from_result(score_list)
        if len(confuse_score_list) > 1:
          logger.info(f'Confuse score: {confuse_score_list}')
        return confuse_score_list[0][1]
            
    def _get_score(self, raw, s):
      score = 0
      
      if len(s.strip())  == 0:
        return score
      s =  self._nomalize_word(s)
      split_s_to_set = set(s.split())
      if len(split_s_to_set) > 2 and s in raw:
          return 5
      
      for w in s.split():
          if w in raw:
              score += 1
      return score

    def _nomalize_word(self, s):
        return unidecode(s).lower()



def remove_word_from_left(input_string):
    return ' '.join(input_string.split()[1:])

class TrieNode:
    node_count = 0

    def __init__(self):
        self.word = None
        self.province = None
        self.district = None
        self.ward = None
        self.children = {}
        TrieNode.node_count += 1

    def insert(self, word_set):
        node = self
        word, ward, district, province = word_set
        reversed_word = unidecode(word[::-1].lower().replace('.', ''))
        # reversed_word = unidecode(' '.join([ward, district, province]).lower().replace('.', ''))


        for letter in reversed_word:
            if letter not in node.children:
                node.node_count += 1
                node.children[letter] = TrieNode()
            node = node.children[letter]

        node.word = word
        node.province = province
        node.district = district
        node.ward = ward

    def print(self, level=0):
        if self.word:
            print(f"{' ' * level}Word: {self.word}, Province: {self.province}, District: {self.district}, Ward: {self.ward}")

        for letter, child_node in self.children.items():
            print(f"{' ' * level}Letter: {letter}")
            child_node.print(level + 2)

    def build(self, word_sets):
        words_count = 0
        build_start_time = time.time()

        for word_set in word_sets:
            self.insert(word_set)
            word, ward, district, province = word_set
            
            self.insert((' '.join([province]), "", "", province))
            self.insert((' '.join([district, province]), "", district, province))

            self.insert((' '.join([ward, province]), ward, "", province))
            self.insert((' '.join([ward, district]), ward, district, ""))

            
            words_count += 1

        print(f"Build {words_count} input into {TrieNode.node_count} nodes in {time.time() - build_start_time} seconds")


class EditDistance:
    def __init__(self, word, t_start_search):
        self.word = word
        self.word_length = len(word)
        self.word_reversed = word[::-1]
        self.current_row = range(self.word_length + 1)
        self.t_start_search = t_start_search

    def search(self, node, max_cost):
        results = []
        for letter in node.children:
            if self.search_recursive(node.children[letter], letter, self.word_reversed, self.current_row, results, max_cost) == 0:
              return results
            # self.search_recursive(node.children[letter], letter, self.word_reversed, self.current_row, results, max_cost)
        return results

    def _should_timeout(self):
        t = (time.perf_counter_ns() - self.t_start_search)  / 1_000_000_000
        return t > 0.0969

    def search_recursive(self, node, letter, word, previous_row, results, max_cost):
        if self._should_timeout():
            # raise TimeoutError
            return results
        columns = len(word) + 1
        current_row = [previous_row[0] + 1]
        for column in range(1, columns):
            insert_cost = current_row[column - 1] + 1
            delete_cost = previous_row[column] + 1
            if word[column - 1] != letter:
                replace_cost = previous_row[column - 1] + 1
            else:
                replace_cost = previous_row[column - 1]
            current_row.append(min(insert_cost, delete_cost, replace_cost))

        # if current_row[-1] <= max_cost and node.word is not None:
        #   if not results or current_row[-1] < results[0][0]:
        #       results.clear()
        #       results.append((current_row[-1], (node.province, node.district, node.ward)))
        if current_row[-1] == 0 and node.word is not None:
            heapq.heappush(results, (current_row[-1], (node.province, node.district, node.ward)))
            return 0

        if current_row[-1] <= max_cost and node.word is not None:
            heapq.heappush(results, (current_row[-1], (node.province, node.district, node.ward)))

        if min(current_row) <= max_cost:
            for letter in node.children:
                if self.search_recursive(node.children[letter], letter, word, current_row, results, max_cost) == 0:
                  return 0

        # return results
        return min(current_row)


class Input:
    def __init__(self, input_str):
        self.input_str = input_str
        self.special_chars = [';', ':', '!', "*", "(", ")", "[", "]", "{", "}", "\\", "|", "/",
                              "<", ">", "?", "@", "#", "$", "%", "^", "&", "_", "=", "+", "~", "`", "-", ","]

        self.unit_strings = ['Thành Phố', 'Thành Fhố', 'Thành Phô', 'T Phố', 'T. Phố', 'T. Phw', 'Tỉnh', 'Tỉnwh',
                             'Quận', 'Huyện', 'Huyen', 'Huyên', 'Hyện', 'HZuyện',
                             'Thị Xã', 'Thị Trấn', 'Phường', 'Xã', 'T. P ', 'TP.', 'TP ', 'TT ', 'TX ', 'Tt.', 'T. X.',
                             'TTr. ', 'T. X. ', 'TX.',
                             'Tp ', 'Tp. ', 'Tt ', 'Tx ', 'T.', 'Q.', 'H.', 'P.', ' P ', 'X.', 'G.', ' H ', ' X ',
                             ' T ', ' Q ', 'F']

        self.cut_strings = ['Số Nhà', 'Số ', 'Ngõ ', 'Ngách ', 'Thôn ', 'Ấp ',
                            'Tổ ', 'Khu Phố', 'Khu ', 'TDP', 'DTM', 'Cụm', 'ĐTM']
        self.raw_input = []

    def add_space_before_uppercase(self):
        result = ''
        for i, char in enumerate(self.input_str):
            if i != 0 and char.isupper() and self.input_str[i - 1] != ' ' and not self.input_str[i - 1].isupper():
                result += ' ' + char
            else:
                result += char
        self.input_str = result

    def capitalize_first_letters(self):
        words = self.input_str.split(' ')
        capitalized_words = [word[:1].upper() + word[1:] for word in words]
        self.input_str = ' '.join(capitalized_words)

    def remove_inner_numbers(self):
        words = self.input_str.split()
        result = []
        for word in words:
            cleaned_word = re.sub(r'(?<=\D)\d+(?=\D)', '', word)
            result.append(cleaned_word)
        self.input_str = ' '.join(result)

        if re.search(r'\d+', self.input_str):
            if int(re.search(r'\d+', self.input_str).group(0)) > 21:
                digit = re.search(r'\d+', self.input_str).group(0)
                self.input_str = self.input_str[self.input_str.find(digit) + len(digit) + 1:]
                
                # self.input_str = self.input_str[self.input_str.find(' ') + 1:]
                self.input_str = self.input_str.strip()

    def clean_noise_input(self):
        words = [word.strip() for word in self.input_str.split(',')]
        # self.input_str = ', '.join([word for word in words[-3:]])
        words = list(filter(lambda word: len(word) > 0, words))
        if len(words) > 3:
          self.input_str = ' '.join([word for word in words[-3:]])
        else:
          self.input_str = ' '.join(words)
        
        for char in self.special_chars:
            self.input_str = self.input_str.replace(char, '')
        
        # self.input_str = ''.join(char for char in self.input_str if char not in self.special_chars)
        
        self.remove_inner_numbers()
        
        self.capitalize_first_letters()
        self.add_space_before_uppercase()
        
        pattern = re.compile(r'.*?((?<!\w)P\.|Phường .+|Thị Trấn .+|TT .+|TT\.|(?<!Thị )Xã.+|(?<![T,. ])X\.+)')

        # Search for any of the combined patterns
        match = pattern.search(self.input_str)
        if match:
            self.input_str = self.input_str[match.start(1):]
        
        for string in self.unit_strings:
            self.input_str = self.input_str.replace(string, ' ')

        for string in self.cut_strings:
            if string in self.input_str:
                self.input_str = self.input_str[self.input_str.find(string) + len(string) + 1:]
                self.input_str = self.input_str[self.input_str.find(' ') + 1:]

        while '  ' in self.input_str:
            self.input_str = self.input_str.replace('  ', ' ')

        self.input_str = self.input_str.replace('. ', '.')

        self.input_str = self.input_str.strip().strip('.')

        # return self.input_str
        return  unidecode(self.input_str.lower())

In [5]:
# test preprocess input
test_case = [
  ("107 Quang Trung Nam định-T.nam định", "quang trung nam dinh nam dinh"),
]

for t in test_case:
  processed_input = Input(t[0]).clean_noise_input()
  if processed_input != t[1]:
    print(f'Raw {t[0]}, processed: {processed_input}')
    raise Exception(f'Check your logic')

In [6]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
# !rm -rf test.json
# this link is public test
# !noglob gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json
# !noglob gdown --fuzzy https://drive.google.com/file/d/1-gQ0itfxRfbPlEglh1e2M8AlK-6YMMgA/view?usp=sharing -O test.json

In [7]:
# NOTE: DO NOT change this cell
# This cell is for scoring

TEAM_NAME = 'Group4'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json') as f:
    data = json.load(f)

summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]

    ok = 0
    try:
        start = time.perf_counter_ns()
        result = solution.process(address)
        answer = data_point["result"]
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        ok += int(answer["province"] == result["province"])
        ok += int(answer["district"] == result["district"])
        ok += int(answer["ward"] == result["ward"])
        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            int(answer["province"] == result["province"]),
            answer["district"],
            result["district"],
            int(answer["district"] == result["district"]),
            answer["ward"],
            result["ward"],
            int(answer["ward"] == result["ward"]),
            ok,
            timer[-1] / 1_000_000_000,
        ])
    except Exception as e:
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        # any failure count as a zero correct
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")


print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_correct',
    'district',
    'district_student',
    'district_correct',
    'ward',
    'ward_student',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns

print(f'{TEAM_NAME = }')
print(f'{EXCEL_FILE = }')
print(df2)

!pip install xlsxwriter
writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
df2.to_excel(writer, index=False, sheet_name='summary')
df.to_excel(writer, index=False, sheet_name='details')
writer.close()


Build 79662 input into 637967 nodes in 6.668400049209595 seconds
------------------------------
TEAM_NAME = 'Group4'
EXCEL_FILE = 'Group4.xlsx'
   correct  total  score / 10  max_time_sec  avg_time_sec
0     1277   1371        9.31        0.0978        0.0309
